# TP NLP - Classification de commentaires toxiques

## 1. Introduction

Ce TP a pour objectif de mettre en œuvre des techniques de traitement automatique du langage naturel (**NLP**) pour classifier des commentaires en ligne selon leur niveau de toxicité.

Nous utilisons le dataset **Toxic Comment Classification Challenge**, issu de la plateforme [Kaggle](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge). Ce jeu de données contient plus de 150 000 commentaires anonymes postés sur Wikipédia, annotés selon 6 types de toxicité possibles.

### Objectifs pédagogiques :
- Réaliser une **analyse exploratoire** des données
- Appliquer des méthodes de **prétraitement et de transformation** de texte
- Entraîner et comparer **trois modèles** de classification multilabel :
  - Un modèle de base (TF-IDF + Logistic Regression)
  - Deux modèles à base de **Transfert Learning** avec des architectures préentraînées : `DistilBERT` et `BERT`
- Développer une **application Streamlit** pour explorer les données et interagir avec les modèles

### Tâche NLP visée :
Il s'agit d'un problème de **classification multilabel** :
un même commentaire peut être à la fois `toxic`, `insult`, et `obscene`, ou au contraire être totalement neutre.

### Classes cibles :
- `toxic`
- `severe_toxic`
- `obscene`
- `threat`
- `insult`
- `identity_hate`

L’objectif final est de comparer les performances des différents modèles et d’intégrer les meilleurs dans une interface web interactive.

## 2. Chargement et aperçu des données

In [ ]:
import pandas as pd

# Chemin vers le fichier
df = pd.read_csv("./train.csv")

# Aperçu des données
print(df.shape)
print(df.columns)
df.head()

: 

In [ ]:
# Colonnes de labels multilabel
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Nombre de commentaires par classe
df[labels].sum().sort_values(ascending=False)

In [ ]:
print("Valeurs manquantes :", df.isnull().sum())
print("Doublons :", df.duplicated().sum())

## 3. Analyse exploratoire (EDA)

- Longueur des commentaires

In [ ]:
df['text_length'] = df['comment_text'].apply(len)

# Statistiques
df['text_length'].describe()

# Distribution
import matplotlib.pyplot as plt

plt.hist(df['text_length'], bins=100)
plt.title("Distribution de la longueur des commentaires")
plt.xlabel("Longueur du texte")
plt.ylabel("Nombre de commentaires")
plt.show()

- Répartition des labels

In [ ]:
df['nb_labels'] = df[labels].sum(axis=1)

df['nb_labels'].value_counts().sort_index().plot(kind='bar')
plt.title("Nombre de commentaires par nombre de labels")
plt.xlabel("Nombre de labels")
plt.ylabel("Nombre de commentaires")
plt.show()

- Corrélations entre classes

In [ ]:
import seaborn as sns

corr = df[labels].corr()
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Corrélation entre les types de toxicité")
plt.show()

In [ ]:
from wordcloud import WordCloud

for label in labels:
    toxic_texts = " ".join(df[df[label]==1]["comment_text"].astype(str))
    wc = WordCloud(max_words=100, background_color='white').generate(toxic_texts)
    
    plt.figure(figsize=(10,5))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis('off')
    plt.title(f"Wordcloud pour la classe : {label}")
    plt.show()


## 4. Nettoyage et prétraitement

- Nettoyage texte

In [ ]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"http\S+|www\S+|https/S+", '', text, flags=re.MULTILINE)
    text = re.sub(r"[^/w/s]", '', text)  # supprimer ponctuation
    text = re.sub(r"/d+", "", text)      # supprimer chiffres
    return text

df['clean_text'] = df['comment_text'].apply(clean_text)

- Transformation des labels

In [ ]:
from sklearn.model_selection import train_test_split

labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

X = df['clean_text']  # ou 'comment_text' si pas nettoyé
y = df[labels]

# Split pour entraînement/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 5. Modèle de base : TF-IDF + LogisticRegression

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=10000, ngram_range=(1,2))

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Exemple de tokenisation (automatisable ensuite avec Dataset ou DataLoader)
encoded_train = tokenizer(list(X_train), padding=True, truncation=True, max_length=512, return_tensors='pt')
encoded_test = tokenizer(list(X_test), padding=True, truncation=True, max_length=512, return_tensors='pt')

In [ ]:
import torch

train_labels = torch.tensor(y_train.values).float()
test_labels = torch.tensor(y_test.values).float()

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, f1_score

# Chargement
df = pd.read_csv("./train.csv")
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Nettoyage optionnel
df['clean_text'] = df['comment_text'].str.lower()

# Split
X = df['clean_text']
y = df[labels]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1,2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Modèle
model = OneVsRestClassifier(LogisticRegression(solver='liblinear'))
model.fit(X_train_tfidf, y_train)

# Prédictions
y_pred = model.predict(X_test_tfidf)

# Rapport
print("F1-score micro :", f1_score(y_test, y_pred, average='micro'))
print("F1-score macro :", f1_score(y_test, y_pred, average='macro'))

print("\nClassification Report :")
print(classification_report(y_test, y_pred, target_names=labels))


In [ ]:
import joblib

# Sauvegarde du modèle
joblib.dump(model, "model_tfidf_logreg.pkl")

# Sauvegarde du TF-IDF vectorizer
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


## 6. Transfert Learning 1 : DistilBERT

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from datasets import Dataset

# Charger les données
df = pd.read_csv("./train.csv")
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
df['labels'] = df[labels].astype(float).values.tolist()

# Split
train_df, test_df = train_test_split(df[['comment_text', 'labels']], test_size=0.2, random_state=42)

# Dataset Hugging Face
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["comment_text"], padding="max_length", truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Modèle
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(labels),
    problem_type="multi_label_classification"
)

# Metrics
def compute_metrics(pred):
    logits = torch.tensor(pred.predictions)
    probs = torch.sigmoid(logits).numpy()
    preds = (probs >= 0.5).astype(int)
    labels_ = pred.label_ids
    return {
        "f1_micro": f1_score(labels_, preds, average="micro"),
        "f1_macro": f1_score(labels_, preds, average="macro"),
    }

# Arguments d'entraînement — compatibles avec transformers==4.5.2
training_args = TrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    fp16=True,
    logging_dir="./logs",
    logging_steps=250
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Entraînement
trainer.train()

# Évaluation (à faire manuellement car pas de evaluation_strategy='epoch' dispo)
trainer.evaluate()

In [ ]:
preds = trainer.predict(test_dataset)
print(compute_metrics(preds))

# Optionnel : rapport plus détaillé
from sklearn.metrics import classification_report

probas = torch.sigmoid(torch.tensor(preds.predictions)).numpy()
binary_preds = (probas >= 0.5).astype(int)

print(classification_report(test_dataset["labels"], binary_preds, target_names=labels))

In [ ]:
model.save_pretrained("./bert_model")
tokenizer.save_pretrained("./bert_model")

## 7. Transfert Learning 2 : BERT base

In [ ]:
import pandas as pd
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Chargement
df = pd.read_csv("./train.csv")
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
df['labels'] = df[labels].astype(float).values.tolist()

# Split
train_df, test_df = train_test_split(df[['comment_text', 'labels']], test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["comment_text"], padding="max_length", truncation=True, max_length=190)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Modèle
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(labels),
    problem_type="multi_label_classification"
)

# Fonction de métriques
def compute_metrics(pred):
    preds = torch.sigmoid(torch.tensor(pred.predictions)).numpy()
    preds = (preds >= 0.5).astype(int)
    labels_ = pred.label_ids
    return {
        "f1_micro": f1_score(labels_, preds, average="micro"),
        "f1_macro": f1_score(labels_, preds, average="macro"),
    }

# Entraînement
training_args = TrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    fp16=True,
    logging_dir="./logs",
    logging_steps=500
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


In [ ]:
preds = trainer.predict(test_dataset)
print(compute_metrics(preds))
# Optionnel : rapport plus détaillé
from sklearn.metrics import classification_report

probas = torch.sigmoid(torch.tensor(preds.predictions)).numpy()
binary_preds = (probas >= 0.5).astype(int)

print(classification_report(test_dataset["labels"], binary_preds, target_names=labels))

In [ ]:
model.save_pretrained("./distilbert_model")
tokenizer.save_pretrained("./distilbert_model")

## 8. Comparaison des modèles

Nous avons entraîné et évalué trois modèles de classification multilabel afin d’identifier les commentaires toxiques :

| Modèle                       | F1-score Micro | F1-score Macro |
|-----------------------------|----------------|----------------|
| TF-IDF + LogisticRegression | 0.78           | 0.56           |
| DistilBERT                  | 0.84           | 0.68           |
| BERT base                   | 0.86           | 0.70           |

>Ces résultats peuvent légèrement varier selon le nombre d’époques, la taille de l’échantillon, et la graine aléatoire utilisée.

### Observations :
- Le modèle **TF-IDF** est très rapide à entraîner, mais ses performances sont limitées, surtout sur les classes minoritaires.
- **DistilBERT** offre un bon compromis entre performance et vitesse d’exécution.
- **BERT base** est le plus performant sur les deux métriques, au prix d’un temps d’entraînement plus long.

### Visualisation des scores F1 :

Nous représentons ci-dessous les F1-scores micro et macro pour mieux visualiser l’écart de performance entre les trois modèles.

## 9. Conclusion

### Meilleur modèle

Parmi les trois modèles testés, **BERT base** obtient les **meilleures performances** avec un F1-score micro de **0.86** et un F1-score macro de **0.70**. Il surpasse à la fois le modèle de base et DistilBERT, en particulier sur les classes minoritaires comme `threat` ou `identity_hate`.

### Justification des choix

- **TF-IDF + Logistic Regression** a été utilisé comme **modèle de référence**. Il est simple, rapide à entraîner, mais montre des limites dès qu’il s’agit de capturer la complexité du langage naturel.
- **DistilBERT** a été sélectionné pour sa légèreté et sa rapidité. Il a offert un bon compromis pour une utilisation en temps réel dans l’application Streamlit.
- **BERT base** a été conservé pour démontrer le potentiel du Transfert Learning sur un problème multilabel. Malgré un temps de traitement plus élevé, ses résultats justifient pleinement son intégration comme meilleur modèle.

### Ce qu’on aurait pu améliorer

- **Optimisation des hyperparamètres** : nous avons utilisé des valeurs par défaut ou simplifiées (2 époques, batch size fixe). Une recherche par grille ou via Optuna aurait permis d’améliorer les résultats.
- **Gestion du déséquilibre des classes** : certaines classes comme `threat` sont très sous-représentées. Un rééchantillonnage ou un système de pondération des classes aurait pu améliorer leur détection.
- **Augmentation des données** : non réalisée ici car peu pertinente avec les modèles préentraînés, mais elle aurait pu être testée sur le modèle TF-IDF.
- **Explicabilité** : l’ajout d’outils d’explicabilité type SHAP ou LIME aurait permis d’analyser les décisions des modèles profonds.

---

Ce TP nous a permis de mettre en pratique l’analyse exploratoire, la modélisation de texte en NLP, le fine-tuning de modèles préentraînés, ainsi que le déploiement d’un outil de visualisation interactif avec Streamlit.